In [22]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


# Bibliotecas


In [3]:
!pip install langchain_community
!pip install langchain_openai
!pip install pypdf
!pip install rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00


###Leitura do arquivo

o arquivo foi o Diário Oficial da União do dia 13/05/2024, o dia onde falaram sobre o ENEM 2024

In [24]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.2 MB/s eta 0:00:00


In [18]:
from langchain_community.document_loaders import PyPDFLoader

# Carrega o arquivo PDF
loader = PyPDFLoader(
    file_path = "/content/2024_05_13_ASSINADO_do3.pdf",
    extract_images = True
)

docs = []
docs_lazy = loader.lazy_load()


for doc in docs_lazy:
    docs.append(doc)



REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACIONAL
Ano CLXII Nº 91 Brasília - DF, segunda-feira, 13 


In [33]:
print(docs[0].page_content[:150])

REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACIONAL
Ano CLXII Nº 91 Brasília - DF, segunda-feira, 13 de maio de 2024ISSN 
1677-7069
3
Documento assinad


### Dividindo Documentos - Splitting/Chunking

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

4585

In [20]:
print(all_splits[2].page_content)

Ministério dos Direitos Humanos e da Cidadania ................................................................ 
21
Ministério da Educação ........................................................................................................... 22
Ministério do Empreendedorismo, da Microempresa e da Empresa de Pequeno Po rte .. 74
Ministério do Esporte ............................................................................................................. 74
Ministério da Fazenda ............................................................................................................. 75
Ministério da Gestão e da Inovação em Serviços Públicos ................................................. 80
Ministério da Integração e do Desenvolvimento Regional .................................................. 88
Ministério da Justiça e Segurança Pública ............................................................................ 91


### Indexando - Store

In [25]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [26]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Quando será realizado o ENEM 2024?")

len(retrieved_docs)

6

### Buscando e Recuperando Informações - Retrieve

In [27]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

### Gerando Respostas - Generate


In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [32]:
for chunk in rag_chain.stream("Quando será realizado o ENEM 2024?"):
    print(chunk, end="", flush=True)

O ENEM 2024 será realizado nos dias 3 e 10 de novembro de 2024. As inscrições ocorrerão de 27 de maio a 7 de junho de 2024.